# Assignment 2. Econophysics simulator

Economic inequality is one of the defining social issues of our age. Yet we have a poor grasp of the scale of inequality,
as [described in Scientific American](https://www.scientificamerican.com/article/economic-inequality-it-s-far-worse-than-you-think/) 
and nicely shown in [this video](https://www.youtube.com/watch?v=QPKKQnijnsM):
[<img src="res/inequality.png" style="height:16em">](https://www.youtube.com/watch?v=QPKKQnijnsM)

How does inequality arise? Is it an inevitable outcome of liberal economics, and if so how can it be  mitigated by economic policy? These questions [have been studied by economists](https://link.springer.com/article/10.1140/epjst/e2016-60162-3) and more recently 
[by](https://phys.org/news/2007-04-world-economies-similarities-economic-inequality.html)
[physicists](https://arxiv.org/abs/1606.06051).
In this assignment you will investigate a simple "econophysics" model of inequality. The notes for [&sect;2](2.%20Numerical%20computation.ipynb) suggest ways to take the investigation further.

<div class="alert alert-warning">**About this assignment:** This assignment tests your vectorized thinking. You will be asked to run simulations on a population of hundreds of thousands of individuals, over many timesteps. Your code _must_ use NumPy vectorized operations rather than iterating over the population. You may use Python iteration over timesteps.
You can organize your code however you like. Please create a new notebook for your answers to this assignment.
</div>

# Part A: kinetic exchange model
This section is worth 1 mark. Check your answers as described in 
[&sect;0.3](0.%20About%20this%20course.ipynb#grader) using `section='assignment2a'`.

Here is a simple model. There are $N$ individuals in the population, each with an initial wealth of &pound;1. Every timestep, we randomly group them into $N/2$ pairs. (Assume $N$ is even.) For every pair, we simulate an economic exchange, as follows. Let the two paired individuals have wealth $v$ and $w$, and update their wealth according to
$$
v_{\text{new}} = R(v+w),
\quad
w_{\text{new}} = (1-R)(v+w)
$$
where $R$ is a random number in $[0,1]$, chosen independently for every pair and at every timestep.
This model is loosely inspired by the physics of gases, in which two gas molecules exchange a random amount of energy whenever they collide.

We can measure inequality with the [Gini coefficient](https://en.wikipedia.org/wiki/Gini_coefficient),
$$
G = 2\frac{\sum_{i=1}^N i\, w_{(i)}}{N \sum_i w_{(i)}} - \Big(1 + \frac{1}{N}\Bigr)
$$
where $w_{(1)}$ is the smallest value, $w_{(2)}$ the second smallest etc. If everyone has the same wealth then $G=0$; if one person has all the wealth then $G=1-1/N$.

**Question 1.** The model needs us to randomly group the population into $N/2$ pairs. 
We can do this by randomly permuting the vector $[0,...,N-1]$, letting the vector `m1` consist of the first $N/2$ integers and `m2` consist of the rest, and interpreting it as "`m1[i]` is paired with `m2[i]`".

Write a function `pairs(N)` that returns a tuple `(m1,m2)` where `m1` and `m2` are both vectors of length $N/2$ as described above. For example, if you run `pairs(6)`, you might get the output

> `(array[(3, 0, 1]), array([2, 4, 5]))`

```
# Submitting your answer:
q = GRADER.fetch_question('q1')
m1,m2 = pairs(q.n)
ans = {'n': len(np.unique(np.concatenate([m1,m2]))), 's': np.std(np.abs(m1-m2))}
GRADER.submit_answer(q, ans)
```

**Question 2.** Write a function `kinetic_exchange(v,w)` which takes two wealth vectors `v` and `w`, each of length $N/2$, and returns a tuple `(vnew, wnew)` with two new vectors, according to the kinetic exchange model.
```
# Submitting your answer:
q = GRADER.fetch_question('q2')
v,w = np.linspace(1,5,q.n), np.linspace(1,2,q.n)**q.p
vnew,wnew = kinetic_exchange(v,w)
ans = {'m1': np.mean(vnew), 's2': np.std(wnew)}
GRADER.submit_answer(q, ans)
```

**Question 3.** Write a function `gini(w)` which takes a vector `w` and returns the Gini coefficient.
```
# Submitting your answer:
q = GRADER.fetch_question('q3')
w = np.linspace(0,1,q.n)**q.p
g = gini(w)
GRADER.submit_answer(q, {'g': g})
```

**Question 4.** Write a function `sim(N, T)` which runs the kinetic exchange model on a population of $N$ individuals for $T$ timesteps. It should return a pair `(w, gs)` where `w` is the wealth vector after $T$ timesteps, and `gs` is a length $T$ vector where `gs[i]` is the Gini coefficient at timestep $i$.
```
# Submitting your answer:
q = GRADER.fetch_question('q4')
w,gs = sim(q.n, q.t)
ans = {'gm': np.mean(gs[int(q.t/2):]), 'gs': np.std(gs[int(q.t/2):]), 'ws': np.std(w)}
GRADER.submit_answer(q, ans)
```

**Question 5.** Simulate a population of 500,000 over 30 iterations. Plot the Gini coefficient as a function of timestep. (You don't have to submit your plot, but it may be assessed in the ticking session.) Your plot should look something like this:
![gini coefficient](res/ass2_q5.png)

# Part B: economic mobility
This section is worth 1 mark. Check your answers as described in 
[&sect;0.3](0.%20About%20this%20course.ipynb#grader) using `section='assignment2b'`.

Some degree of inequality might be acceptable if economic mobility were high, i.e. if everyone had similar chances of reaching either end of the wealth distribution. Economic mobility is often measured by splitting the population into five equal brackets, and measuring the chance of moving between brackets. From the [Wikipedia article on economic mobility](https://en.wikipedia.org/wiki/Economic_mobility):

> in terms of relative mobility [a report](https://www.brookings.edu/research/economic-mobility-of-families-across-generations/) stated: "contrary to American beliefs about
> equality of opportunity, a child’s economic position is heavily influenced by that of his
> or her parents." 42% of children born to parents in the bottom fifth of the income
> distribution ("quintile") remain in the bottom, while 39% born to parents in the top fifth
> remain at the top.

Let's measure economic mobility by recording the wealth distribution at one timepoint, and again some number of timesteps later, splitting the two distributions into quintiles, and counting what fraction of the population moved by more than one quintile from beginning to end. (In each timestep a median individual might find their wealth increasing or decreasing by around 50%, so one timestep corresponds roughly to several years of human life.) For example, if we have a population of 5000 and we draw up a matrix $A$ where $A_{i j}$ is the number of people who start in quintile $i$ and end up in quintile $j$, we might get
$$
A = \left( \begin{matrix}
344& 313& 243& 100&   0\\
266& 261& 302& 167&   4\\
212& 260& 225& 272&  31\\
147& 143& 183& 331& 196\\
 31&  23&  47& 130& 769
\end{matrix} \right)
$$
(A quick check: the row sums and column sums are all 1000.) The number who moved by more than one quintile is 1148, which is 23% of the population.

**Question 6.** In a perfectly mobile economy, where everyone has equal chance of reaching any quintile, what fraction of people are expected to move by more than one quintile?
```
q = GRADER.fetch_question('q6')
GRADER.submit_answer(q, your_answer)
```

**Question 7.** Write a function `mobility(v,w)` that returns the proportion of people who moved by more than one quintile, where `v[i]` and `w[i]` measure respectively the wealth of individual $i$ at the beginning and end of a time period. Hint: look up [`np.percentile`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.percentile.html#numpy.percentile) and [`np.digitize`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.digitize.html). 
```
# Submitting your answer:
q = GRADER.fetch_question('q7')
v,w = np.arange(q.n)**q.a, np.arange(q.n)**q.a * np.random.random(q.n)
GRADER.submit_answer(q, mobility(v,w))
```

**Question 8.** Simulate the kinetic exchange model long enough for it to stabilize, and measure the wealth vector `w0`. Run it $t$ timesteps further, and find the wealth vector `v`, and compute `mobility(w0,v)`. It's up to you to decide how to judge stabilization; you don't have to explain your method but you do have to submit a correct answer.
```
# Submitting your answer:
q = GRADER.fetch_question('q8')
# For a population size q.n, measure mobility over q.t timesteps
GRADER.submit_answer(q, your_answer)
```

**Question 9.** Compute the social mobility for a population of 500,000 as in Question 8, over a sequence of timesteps. Plot a graph of economic mobility as a function of time. (You don't have to submit your plot, but it may be assessed in the ticking session.) Your plot should look something like this:
![mobility](res/ass2_q9.png)